In [2]:
from elasticsearch import Elasticsearch
from elasticsearch.exceptions import ConnectionError, NotFoundError
from elasticsearch.helpers import bulk
import json
import pandas as pd
import numpy as np
import string
import random
from datasets import load_dataset
dataset = load_dataset("json", data_files=".\corpus.json")

c:\Users\ifeol\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration default-380d03813620ed62
Found cached dataset json (C:/Users/ifeol/.cache/huggingface/datasets/json/default-380d03813620ed62/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|██████████| 1/1 [00:00<00:00, 16.39it/s]


In [15]:
dataset['train'][3]['products'][0]['languages']['en']

In [ ]:
def unique_id(size):
    chars = list(set(string.ascii_lowercase + string.digits).difference('LIO01'))
    return ''.join(random.choices(chars, k=size))


text = []
uid = []

for i in range(10):
    for j in range(len(dataset['train'][i]['products'])):
        if dataset['train'][i]['products'][j]['languages']['en'] is not None:
            if len(dataset['train'][i]['products'][j]['languages']['en']) != 0:
                for k in range(len(dataset['train'][i]['products'][j]['languages']['en'])):
                    paragraphText = dataset['train'][i]['products'][j]['languages']['en'][k]['paragraphText']
                    id = dataset['train'][i]['manufacturer'] + '_' + unique_id(5)
                    text.append(paragraphText)
                    uid.append(id)
                    
list_ = [uid, text]
data = pd.DataFrame(list_).T
data.columns= ['uid', 'text']

In [4]:
from elastic import Elastic
elastic = Elastic()

In [ ]:
elastic.create_index(name="coffee", config_file="./index_config.json")

In [5]:
from embedders import SwivelEmbedder, BertEmbedder
bert = BertEmbedder()
swivel = SwivelEmbedder()

data['swivel_embedding'] = data['text'].apply(lambda t: swivel.embed_single_text(t)[0].tolist())
data['bert_embedding'] = data['text'].apply(lambda t: bert.embed_single_text(t)[0].tolist())

Loaded pre-trained model https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1 successfully!
Loaded pre-trained model https://tfhub.dev/tensorflow/cord-19/swivel-128d/3 successfully!


In [6]:
data['_op_type'] = "index"
data['_index'] = "coffee"
data = data[['_op_type', '_index', 'uid', 'swivel_embedding', 'bert_embedding']]
index_data = data.to_dict(orient='records')
print(len(index_data))
print(index_data[0])

15
{'_op_type': 'index', '_index': 'coffee', 'uid': 'RussellHobbs_xok37', 'swivel_embedding': [-0.346613834746382, 0.43383212806710525, 0.09046593967231623, 0.2010883789415725, 0.17466118542698506, -0.3548303535637184, -0.12373643250010927, 0.3215158644417337, 0.2592890740009963, -0.10295941841778494, 0.20847201664494255, -0.36678626233437883, -0.025673691142456722, -0.3396039323095407, 0.8504860767647274, -0.08153197850005323, 0.6150776049893185, 0.8861813859550424, -0.06888744746297251, -1.1687792345466417, -0.013225535377880284, -0.2481299533468827, -0.24317668110825855, 0.1866749964905303, -0.18321621789026216, 0.7506048365161123, -0.07479529994510403, 0.05845114860859088, -0.27582235459983184, -0.17007055748068411, -0.6834723885836642, -0.7658263594923637, 0.8414153634788942, -0.16503594611959288, -0.02402869929144451, 0.15156943120663713, -0.45229843344421644, 0.41649930812703606, -0.1877804569003607, 0.3422395573658991, -0.528569283476937, -0.13675972552547788, 0.473228423988175

In [7]:
elastic.index_documents(index_data)

In [8]:
def retrieve(query:str, n_returns:int, index_name: str, embedder='swivel'):
    '''
    return: 2-D list [['uid': cosine_similarity]], sorted by cosine score descendingly
    e.g.[['AEG_h3j5b', 0.963506],
         ['Melitta_u2k5s', 0.9492332000000001],
         ['Jura_q8g4h', 0.9438302999999999]]
    '''
    if embedder == 'swivel':
        query_vector = swivel.embed_single_text(query)[0].tolist()
    elif embedder == 'bert':
        query_vector = bert.embed_single_text(query)[0].tolist()
    else:
        raise ValueError("embedder must be either 'swivel' or 'bert'!")
        
    script_query = {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": f"cosineSimilarity(params.query_vector, '{embedder}_embedding') + 1.0", # Add 1.0 because ES does not support negative score
                "params": {"query_vector": query_vector}
            }
        }
    }
    
    response = elastic.search(index_name, script_query, n_returns, list_fields_to_return=["uid"])
    
    res = []
    for hit in response["hits"]["hits"]:
        res.append([hit["_source"]["uid"], hit["_score"] - 1])
    return np.array(res)

In [ ]:
query = "descaling the machine"

print("10 best matching documents of the query:")
print(retrieve(query=query, n_returns=10, index_name="coffee", embedder='swivel'))